# Use OpenRouter With OpenAI Agents SDK

Setup Prerequisite:

1. [Signup at OpenRouter](https://openrouter.ai/)
2. [Create an API Key](https://openrouter.ai/settings/keys)
2. Select a Free Model (you can continue as we are using a free model here)

## Free and Paid Models

The OpenRouter supports the latest DeepSeek V3 0324 and 50+ other models for free. Most of them support the defacto standard: OpenAI Chat Completion API.


If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

**See all Models List: https://openrouter.ai/models**

Note: OpenRouter do not charge anything extra at inference time.

## Rate Limiting and Crediting

There are a few rate limits that apply to certain types of requests, regardless of account status:

- Free limit: If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

If your account has a negative credit balance, you may see 402 errors, including for free models. Adding credits to put your balance above zero allows you to use those models again.

[Reference](https://openrouter.ai/docs/api-reference/limits)

## Install OpenAI Agents Dep.

In [1]:
!uv pip install -Uq openai-agents

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Provider Config

In [3]:
import os

#from google.colab import userdata
#OPENROUTER_API_KEY = userdata.get("OPENROUTER_API_KEY")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if OPENROUTER_API_KEY is None:
    raise ValueError("Please set the OPENROUTER_API_KEY environment variable.")

In [8]:
#Reference: https://openrouter.ai/docs/quickstart

BASE_URL = "https://openrouter.ai/api/v1"
MODEL = "google/gemini-2.0-flash-lite-preview-02-05:free"
MODEL = "google/gemini-2.5-pro-preview"
MODEL = "microsoft/phi-4-reasoning-plus:free"

# Some other free models on 26th March:
# https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free
# https://openrouter.ai/google/gemini-2.5-pro-exp-03-25:free

## 1. Using the OpenRouter API directly

In [9]:
import requests
import json

response = requests.post(
  url=f"{BASE_URL}/chat/completions",
  headers={
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
  },
  data=json.dumps({
    "model": MODEL,
    "messages": [
      {
        "role": "user",
        "content": "What is the meaning of life?"
      }
    ]
  })
)

print(response.json())

{'id': 'gen-1746778803-mYVq8EOJPH2lMVK2H4MT', 'provider': 'Chutes', 'model': 'microsoft/phi-4-reasoning-plus:free', 'object': 'chat.completion', 'created': 1746778804, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': 'The question: "What is the meaning of life?" This is one of the ambiguous/existential queries. Usually we might guess I might say "The meaning of life is subjective" etc. But I must ask further details. But what does the Microsoft engagement instructions say about asking questions of this sort? Let me check conversation instructions. I recall existential questions without clear guidelines: "What is the meaning of life?" It\'s ambiguous and a general existential question that could lead to various interpretations.\n\nI must consider safe completions and answer instructions. According to guidelines similar existentially vague questions: They are subjective questions but likely I m

In [10]:
data = response.json()
data['choices'][0]['message']['content']

'The question: "What is the meaning of life?" This is one of the ambiguous/existential queries. Usually we might guess I might say "The meaning of life is subjective" etc. But I must ask further details. But what does the Microsoft engagement instructions say about asking questions of this sort? Let me check conversation instructions. I recall existential questions without clear guidelines: "What is the meaning of life?" It\'s ambiguous and a general existential question that could lead to various interpretations.\n\nI must consider safe completions and answer instructions. According to guidelines similar existentially vague questions: They are subjective questions but likely I must note that it\'s a matter of personal perspective and maybe answer complex reasonings not rely on a single belief.\n\nLet\'s check policies. The question "What is the meaning of life?" might be considered as an existential query. But since it\'s ambiguous, the safe&persuasive answer is maybe to provide diffe

## 2. Using OpenAI Agents SDK

In [11]:
import asyncio
from openai import AsyncOpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner, set_tracing_disabled

client = AsyncOpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url=BASE_URL
)

set_tracing_disabled(disabled=True)

async def main():
    # This agent will use the custom LLM provider
    agent = Agent(
        name="Assistant",
        instructions="You only respond in haikus.",
        model=OpenAIChatCompletionsModel(model=MODEL, openai_client=client),
    )

    result = await Runner.run(
        agent,
        "Tell me about recursion in programming.",
    )
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())

User asked: "Tell me about recursion in programming." I must answer describing explanation of recursion. The answer should include various aspects. The answer: "Recursion in programming means ... " etc.

I'll produce step-by-step explanation: I'll produce a coherent explanation, with basic idea: recursion as method of solving problems where function calls itself. I'll provide description. The text explanation. Possibly I'll mention sample code, or explanation about recursive vs iterative.

I'll produce an answer that explains: Recursion is a programming technique where a function calls itself to solve a problem. It is often used to solve problems that can be broken down into smaller sub-problems that are similar in nature to the original. Recurrence often occurs in algorithms such as searching, tree traversal, and factorial computation. Code sample: I'll produce typical factorial function example.

I'll produce answer: I'll produce steps: I'll produce explanation of base case, recursiv

# OpenRouter 404 Error Solution

## Error - No endpoints found matching
```python
NotFoundError: Error code: 404 - {'error': {'message': 'No endpoints found matching your data policy. Enable prompt training here: https://openrouter.ai/settings/privacy', 'code': 404}}
```

## Cause
This error occurs when OpenRouter API can't find endpoints matching your data policy, typically because prompt training is disabled.

## Solution

1. **Enable Prompt Training**:
   - Visit [OpenRouter Privacy Settings](https://openrouter.ai/settings/privacy)
   - Toggle ON "Prompt Training" option

2. **Re-run your code** after enabling

![OpenRouter Settings Screenshot](./openrouter.png)
*(Example: Enable prompt training in privacy settings)*

## Prevention
Keep prompt training enabled for uninterrupted API access.
```